# 🎨 anime-character-generator v1.0
## Stable Diffusion + PyTorch (Colab版・シンプル)

**バージョン**: 1.0  
**説明**: PyTorch + Diffusers の基本的な実装  
**ガイド**: [Day3-4 ブログ記事](https://shion.blog/)  
**コード**: [character_generator_v1.py](../character_generator_v1.py)

このノートブックは最もシンプルな実装版です。ブログで詳細に説明されています。

---

Google Colab 環境で動作確認済みの簡易版です。

In [ ]:
# Step 1: GPU 確認
import torch
print(f"✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Step 2: ライブラリインストール（最新版・競合回避）
!pip install -q --upgrade setuptools wheel
!pip install -q diffusers transformers accelerate safetensors
!pip install -q pillow torchvision
print("✅ Installation complete!")

In [ ]:
# Step 3: インポート
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os
import re

print("✅ All libraries imported!")

In [ ]:
# Step 4: モデルロード
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"📦 Loading model... (device: {device})")

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
    safety_checker=None
)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

print("✅ Model ready!")

In [ ]:
# Step 5: 感情バリエーション生成
import os
os.makedirs("outputs/emotions", exist_ok=True)

emotions = {
    "happy": "happy smile, cheerful, joyful",
    "angry": "angry expression, intense eyes",
    "sad": "sad expression, melancholic",
    "surprised": "surprised expression, wide eyes"
}

base = "1girl, anime character, masterpiece, high quality"
emotion_images = {}

print("\n🎭 GENERATING EMOTIONS...\n")

for emotion_name, emotion_desc in emotions.items():
    prompt = f"{base}, {emotion_desc}"
    print(f"  [{emotion_name.upper()}] Generating...", end="")
    
    with torch.no_grad():
        image = pipe(
            prompt=prompt,
            negative_prompt="low quality, blurry",
            num_inference_steps=20,
            guidance_scale=7.0,
            height=512,
            width=512
        ).images[0]
    
    filepath = f"outputs/emotions/character_{emotion_name}.png"
    image.save(filepath)
    emotion_images[emotion_name] = image
    print(" ✅")

print("\n✅ Emotions complete!")

In [ ]:
# Step 6: スタイルバリエーション生成（16種類）
os.makedirs("outputs/styles", exist_ok=True)

styles = {
    "with_hat": "wearing hat, stylish, fashionable",
    "with_earrings": "wearing earrings, jewelry, elegant",
    "with_makeup": "with makeup, beautiful, glamorous",
    "formal": "wearing formal dress, elegant, professional",
    "casual": "casual outfit, relaxed, friendly",
    "long_hair": "long brown hair, soft flowing hair",
    "blush": "soft blush on cheeks",
    "fireplace": "warm fireplace in background",
    "warm_lighting": "warm ambient lighting, soft orange glow",
    "cozy_room": "cozy indoor setting",
    "bokeh": "cinematic bokeh lights",
    "portrait": "upper body portrait",
    "depth_of_field": "shallow depth of field",
    "high_detail": "highly detailed",
    "soft_shading": "soft anime shading",
    "masterpiece": "masterpiece, best quality"
}

style_images = {}

print("👗 GENERATING STYLES (16 variations)...\n")

for i, (style_name, style_desc) in enumerate(styles.items(), 1):
    prompt = f"{base}, {style_desc}"
    print(f"  [{i:2d}/16] {style_name.upper()}...", end="", flush=True)
    
    with torch.no_grad():
        image = pipe(
            prompt=prompt,
            negative_prompt="low quality, blurry",
            num_inference_steps=20,
            guidance_scale=7.0,
            height=512,
            width=512
        ).images[0]
    
    filepath = f"outputs/styles/character_{style_name}.png"
    image.save(filepath)
    style_images[style_name] = image
    print(" ✅")

print("\n✅ Styles complete!")

In [ ]:
# Step 7: 感情結果をグリッド形式で合成
def get_next_version(base_filename):
    """次のバージョン番号を取得"""
    existing_files = os.listdir("outputs")
    pattern = rf'^{re.escape(base_filename)}_v(\d+)\.png$'
    versions = []
    for fn in existing_files:
        match = re.match(pattern, fn)
        if match:
            versions.append(int(match.group(1)))
    next_version = max(versions) + 1 if versions else 1
    return f"{base_filename}_v{next_version}.png"

def create_grid_composite(images_dict, base_filename, rows=2, cols=2, img_size=512, gap=10):
    """複数画像をグリッドレイアウトで合成"""
    use_images = list(images_dict.items())[:rows*cols]
    
    # キャンバスサイズ計算
    canvas_width = cols * img_size + (cols - 1) * gap + gap * 2
    canvas_height = rows * img_size + (rows - 1) * gap + gap * 2
    
    # キャンバス作成
    canvas = Image.new('RGB', (canvas_width, canvas_height), color='white')
    
    # 画像をペースト
    for idx, (name, img) in enumerate(use_images):
        row = idx // cols
        col = idx % cols
        x = gap + col * (img_size + gap)
        y = gap + row * (img_size + gap)
        resized_img = img.resize((img_size, img_size), Image.Resampling.LANCZOS)
        canvas.paste(resized_img, (x, y))
    
    # 保存
    output_filename = get_next_version(base_filename)
    canvas.save(f"outputs/{output_filename}", quality=95)
    print(f"✅ 感情結果: {output_filename}")
    return canvas

emotion_grid = create_grid_composite(emotion_images, "emotion_results", rows=2, cols=2)
emotion_grid.show()

In [ ]:
# Step 8: スタイル結果をグリッド形式で合成
style_grid = create_grid_composite(style_images, "style_results", rows=2, cols=4)
style_grid.show()

In [ ]:
# Step 9: 完了メッセージ
print("\n" + "="*60)
print("✅ GENERATION COMPLETE!")
print("="*60)
print(f"\n📊 Generated composite images:")
print(f"   - emotion_results_v1.png (2x2 grid)")
print(f"   - style_results_v1.png (2x4 grid)")
print(f"\n📁 Total images: {len(emotion_images) + len(style_images)}")
print(f"\n📥 To download:")
print("   1. Left sidebar → Files icon")
print("   2. outputs/ folder → Right click")
print("   3. Download")
print("\n✨ Success!")